### 데이터 로드

In [ ]:
import pandas as pd
from pathlib import Path

# 정제된 데이터 로드
path = Path("../data/tmdb_cleaned.csv")
df = pd.read_csv(path)

df.shape

### 기본 정보 확인

In [ ]:
df.info()
df.describe(include='all')
df.isna().sum()

### release_date 기반 연도 컬럼 생성

In [ ]:
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df["release_year"].value_counts().sort_index()

### 수치형 기초 분포 확인

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

numeric_cols = ["vote_average", "vote_count", "runtime", "budget", "revenue", "popularity"]

# **오류 해결을 위한 코드: 모든 수치형 컬럼을 숫자로 변환**
# 비교 연산 시 str-int 간의 TypeError를 방지합니다.
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Numeric Variables Distribution', fontsize=16)

for idx, column in enumerate(numeric_cols):
    row = idx // 3
    col_idx = idx % 3
    
    # 로그 스케일이 필요한 변수들
    log_scale_cols = ["vote_count", "runtime", "budget", "revenue", "popularity"]
    
    if column in log_scale_cols:
        # 0보다 큰 값만 선택하고 로그 변환
        data = df[column].dropna()
        data = data[data > 0]
        axes[row, col_idx].hist(np.log10(data), bins=50, edgecolor='black', alpha=0.7)
        axes[row, col_idx].set_title(f'{column} Distribution (log scale)')
        axes[row, col_idx].set_xlabel(f'log10({column})')
    else:
        # 일반 스케일
        axes[row, col_idx].hist(df[column].dropna(), bins=50, edgecolor='black', alpha=0.7)
        axes[row, col_idx].set_title(f'{column} Distribution')
        axes[row, col_idx].set_xlabel(column)
    
    axes[row, col_idx].set_ylabel('Frequency')
    axes[row, col_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 결측치 비율 확인 및 상관관계 분석

In [ ]:
# 데이터 분리 및 저장
df_full = df[df['budget'].notnull() & df['revenue'].notnull()].copy()
df_all = df.copy()

print(f"df_all (for Recommendation): {len(df_all):,} movies")
print(f"df_full (for Box Office Analysis): {len(df_full):,} movies")
print(f"Retained: {len(df_full)/len(df_all)*100:.1f}%")

df_full[['budget', 'revenue', 'vote_average', 'popularity']].describe()

### 상관관계 분석

In [ ]:
# df_full 상관관계 분석
correlation = df_full[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix (Box Office Data)', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# 주요 상관관계 출력
revenue_corr = correlation['revenue'].sort_values(ascending=False)
revenue_corr

### ROI 분석

In [ ]:
# ROI 계산 및 분석
df_full['ROI'] = (df_full['revenue'] - df_full['budget']) / df_full['budget']
df_full['profit'] = df_full['revenue'] - df_full['budget']

# ROI 통계
df_full['ROI'].describe()

# ROI Top 10
top_roi = df_full.nlargest(10, 'ROI')[['title', 'budget', 'revenue', 'ROI', 'vote_average']]
top_roi

# ROI 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROI 히스토그램 (outlier 제거)
roi_filtered = df_full[df_full['ROI'] < df_full['ROI'].quantile(0.95)]
axes[0].hist(roi_filtered['ROI'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('ROI Distribution (Bottom 95%)')
axes[0].set_xlabel('ROI')
axes[0].set_ylabel('Frequency')
axes[0].axvline(roi_filtered['ROI'].median(), color='red', linestyle='--', 
                label=f'Median: {roi_filtered["ROI"].median():.2f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Budget vs Revenue scatter (로그 스케일)
axes[1].scatter(df_full['budget'], df_full['revenue'], alpha=0.5, s=10)
axes[1].plot([df_full['budget'].min(), df_full['budget'].max()], 
             [df_full['budget'].min(), df_full['budget'].max()], 
             'r--', label='Break-even line')
axes[1].set_xlabel('Budget')
axes[1].set_ylabel('Revenue')
axes[1].set_title('Budget vs Revenue (log scale)')
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 장르별 흥행 분석

In [ ]:
# 장르 데이터 전처리
df_full['genre_list'] = df_full['genres'].str.split(', ')

# 각 장르별로 행 확장
genres_exploded = df_full.explode('genre_list')

# 장르별 집계
genre_stats = genres_exploded.groupby('genre_list').agg({
    'revenue': ['mean', 'median', 'count'],
    'budget': 'mean',
    'ROI': ['mean', 'median'],
    'vote_average': 'mean',
    'profit': 'mean'
}).round(2)

genre_stats.columns = ['avg_revenue', 'median_revenue', 'count', 'avg_budget', 
                       'avg_ROI', 'median_ROI', 'avg_rating', 'avg_profit']
genre_stats = genre_stats.sort_values('avg_revenue', ascending=False)

genre_stats.head(15)

# 장르별 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

top_genres = genre_stats.head(10)

# 1. Top 10 장르별 평균 수익
axes[0, 0].barh(top_genres.index, top_genres['avg_revenue']/1e6)
axes[0, 0].set_xlabel('Average Revenue (Million $)')
axes[0, 0].set_title('Top 10 Genres by Average Revenue')
axes[0, 0].invert_yaxis()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. 장르별 ROI
axes[0, 1].barh(top_genres.index, top_genres['median_ROI'])
axes[0, 1].set_xlabel('Median ROI')
axes[0, 1].set_title('Top 10 Genres by Median ROI')
axes[0, 1].invert_yaxis()
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. 장르별 평균 평점
axes[1, 0].barh(top_genres.index, top_genres['avg_rating'])
axes[1, 0].set_xlabel('Average Rating')
axes[1, 0].set_title('Top 10 Genres by Average Rating')
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. 장르별 영화 수
axes[1, 1].barh(top_genres.index, top_genres['count'])
axes[1, 1].set_xlabel('Number of Movies')
axes[1, 1].set_title('Top 10 Genres by Movie Count')
axes[1, 1].invert_yaxis()
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 시계열 트랜드 분석

In [ ]:
# 연도별 트렌드 분석 (1980년 이후)
df_full['release_year'] = pd.to_datetime(df_full['release_date'], errors="coerce").dt.year
df_recent = df_full[df_full['release_year'] >= 1980].copy()

# 연도별 집계
yearly_stats = df_recent.groupby('release_year').agg({
    'revenue': 'mean',
    'budget': 'mean',
    'ROI': 'median',
    'vote_average': 'mean',
    'title': 'count'
}).rename(columns={'title': 'movie_count'})

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. 연도별 평균 수익 추이
axes[0, 0].plot(yearly_stats.index, yearly_stats['revenue']/1e6, marker='o', linewidth=2)
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Average Revenue (Million $)')
axes[0, 0].set_title('Average Revenue Trend (1980-2023)')
axes[0, 0].grid(True, alpha=0.3)

# 2. 연도별 평균 예산 추이
axes[0, 1].plot(yearly_stats.index, yearly_stats['budget']/1e6, marker='o', linewidth=2, color='orange')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Budget (Million $)')
axes[0, 1].set_title('Average Budget Trend (1980-2023)')
axes[0, 1].grid(True, alpha=0.3)

# 3. 연도별 ROI 추이
axes[1, 0].plot(yearly_stats.index, yearly_stats['ROI'], marker='o', linewidth=2, color='green')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Median ROI')
axes[1, 0].set_title('ROI Trend (1980-2023)')
axes[1, 0].grid(True, alpha=0.3)

# 4. 연도별 영화 제작 편수
axes[1, 1].bar(yearly_stats.index, yearly_stats['movie_count'], alpha=0.7)
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Number of Movies')
axes[1, 1].set_title('Movie Production Count (1980-2023)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 주요 통계
df_recent['decade'] = (df_recent['release_year'] // 10) * 10
decade_stats = df_recent.groupby('decade').agg({
    'revenue': 'mean',
    'budget': 'mean',
    'ROI': 'median',
    'vote_average': 'mean',
    'title': 'count'
}).round(2)

decade_stats.columns = ['avg_revenue', 'avg_budget', 'median_ROI', 'avg_rating', 'movie_count']
decade_stats

### 국가별 흥행 분석   

In [ ]:
# 국가 데이터 전처리
df_full['country_list'] = df_full['production_countries'].str.split(', ')

# 각 국가별로 행 확장
countries_exploded = df_full.explode('country_list')

# 국가별 집계
country_stats = countries_exploded.groupby('country_list').agg({
    'revenue': ['mean', 'median', 'count'],
    'budget': 'mean',
    'ROI': ['mean', 'median'],
    'vote_average': 'mean',
    'profit': 'mean'
}).round(2)

country_stats.columns = ['avg_revenue', 'median_revenue', 'count', 'avg_budget', 
                         'avg_ROI', 'median_ROI', 'avg_rating', 'avg_profit']
country_stats = country_stats[country_stats['count'] >= 50]
country_stats = country_stats.sort_values('avg_revenue', ascending=False)

country_stats.head(15)

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

top_countries = country_stats.head(10)

# 1. 국가별 평균 수익
axes[0, 0].barh(top_countries.index, top_countries['avg_revenue']/1e6)
axes[0, 0].set_xlabel('Average Revenue (Million $)')
axes[0, 0].set_title('Top 10 Countries by Average Revenue')
axes[0, 0].invert_yaxis()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. 국가별 ROI
axes[0, 1].barh(top_countries.index, top_countries['median_ROI'], color='green')
axes[0, 1].set_xlabel('Median ROI')
axes[0, 1].set_title('Top 10 Countries by Median ROI')
axes[0, 1].invert_yaxis()
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. 국가별 평균 평점
axes[1, 0].barh(top_countries.index, top_countries['avg_rating'], color='orange')
axes[1, 0].set_xlabel('Average Rating')
axes[1, 0].set_title('Top 10 Countries by Average Rating')
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. 국가별 영화 제작 편수
axes[1, 1].barh(top_countries.index, top_countries['count'], color='purple')
axes[1, 1].set_xlabel('Number of Movies')
axes[1, 1].set_title('Top 10 Countries by Movie Count')
axes[1, 1].invert_yaxis()
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 평점과 흥행의 관계

In [ ]:
# Vote Average vs Revenue 관계 분석
df_full['rating_group'] = pd.cut(df_full['vote_average'], 
                                   bins=[0, 4, 5, 6, 7, 8, 10],
                                   labels=['<4', '4-5', '5-6', '6-7', '7-8', '8+'])

rating_analysis = df_full.groupby('rating_group').agg({
    'revenue': ['mean', 'median', 'count'],
    'budget': 'mean',
    'ROI': 'median',
    'vote_count': 'mean'
}).round(2)

rating_analysis.columns = ['avg_revenue', 'median_revenue', 'count', 'avg_budget', 
                           'median_ROI', 'avg_vote_count']
rating_analysis

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. 평점 vs 수익 scatter (로그 스케일)
axes[0, 0].scatter(df_full['vote_average'], df_full['revenue']/1e6, alpha=0.3, s=20)
axes[0, 0].set_xlabel('Vote Average')
axes[0, 0].set_ylabel('Revenue (Million $, log scale)')
axes[0, 0].set_title('Vote Average vs Revenue')
axes[0, 0].set_yscale('log')
axes[0, 0].grid(True, alpha=0.3)

# 2. 평점 구간별 평균 수익
axes[0, 1].bar(rating_analysis.index.astype(str), rating_analysis['avg_revenue']/1e6)
axes[0, 1].set_xlabel('Rating Group')
axes[0, 1].set_ylabel('Average Revenue (Million $)')
axes[0, 1].set_title('Average Revenue by Rating Group')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. 평점 vs 관객수 scatter (로그 스케일)
axes[1, 0].scatter(df_full['vote_average'], df_full['vote_count'], alpha=0.3, s=20, color='green')
axes[1, 0].set_xlabel('Vote Average')
axes[1, 0].set_ylabel('Vote Count (log scale)')
axes[1, 0].set_title('Vote Average vs Vote Count')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

# 4. 평점 구간별 영화 수
axes[1, 1].bar(rating_analysis.index.astype(str), rating_analysis['count'], color='orange')
axes[1, 1].set_xlabel('Rating Group')
axes[1, 1].set_ylabel('Number of Movies')
axes[1, 1].set_title('Movie Distribution by Rating')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 최종 인사이트 요약 및 추천 전략

In [ ]:
# 최종 분석: 성공 영화의 특징
top_10_revenue = df_full['revenue'].quantile(0.9)
success_movies = df_full[df_full['revenue'] >= top_10_revenue].copy()

print(f"Top 10% Revenue Threshold: ${top_10_revenue/1e6:.1f}M")
print(f"Number of Success Movies: {len(success_movies)}")
print(f"\nAverage Budget: ${success_movies['budget'].mean()/1e6:.1f}M")
print(f"Average Revenue: ${success_movies['revenue'].mean()/1e6:.1f}M")
print(f"Average ROI: {success_movies['ROI'].mean():.2f}")
print(f"Average Rating: {success_movies['vote_average'].mean():.2f}")
print(f"Average Vote Count: {success_movies['vote_count'].mean():.0f}")
print(f"Average Runtime: {success_movies['runtime'].mean():.0f} min")

# 장르 분포
print("\n=== Top Genres in Success Movies ===")
success_genres = success_movies.explode('genre_list')['genre_list'].value_counts().head(10)
print(success_genres)

# 제작 국가
print("\n=== Top Countries in Success Movies ===")
success_countries = success_movies.explode('country_list')['country_list'].value_counts().head(10)
print(success_countries)

# 연도별 트렌드
print("\n=== Success Movies by Decade ===")
success_movies['decade'] = (success_movies['release_year'] // 10) * 10
success_by_decade = success_movies.groupby('decade').size()
print(success_by_decade)

### 감독별 집계

In [ ]:
# 감독-장르 매트릭스 (각 감독이 각 장르에서 만든 작품 수)
director_genre_detail = df_full.explode('genre_list').groupby(['director', 'genre_list']).size().reset_index(name='작품수')
director_genre_pivot = director_genre_detail.pivot(index='director', columns='genre_list', values='작품수').fillna(0)

print("\n\n" + "=" * 120)
print("=== 감독별 장르 매트릭스 (Top 10 감독) ===")
print("=" * 120)
top_10_directors = director_stats.head(10).index
print(director_genre_pivot.loc[top_10_directors].to_string())

# 시대별 활동 (연도 구간별 작품 수)
def get_active_decades(director_name):
    """감독이 활동한 연대"""
    director_movies = df_full[df_full['director'] == director_name]
    decades = director_movies['release_year'].apply(lambda x: f"{int(x//10)*10}s" if pd.notna(x) else None)
    return ', '.join(decades.value_counts().head(3).index.tolist())

director_stats['활동시기'] = director_stats.index.map(get_active_decades)

# 7. 흥행 성공률 - 다양한 기준으로 계산

# 기준 1: ROI 3.0 이상 (투자금 대비 3배 이상 수익)
def get_roi_success_rate(director_name, roi_threshold=3.0):
    """ROI 기준 성공률"""
    director_movies = df_full[df_full['director'] == director_name]
    success_count = (director_movies['ROI'] >= roi_threshold).sum()
    total_count = len(director_movies)
    return round(success_count / total_count * 100, 1) if total_count > 0 else 0

# 기준 2: 수익 1억 달러 이상 (블록버스터 기준)
def get_blockbuster_rate(director_name, revenue_threshold=100_000_000):
    """블록버스터(1억불 이상) 성공률"""
    director_movies = df_full[df_full['director'] == director_name]
    success_count = (director_movies['revenue'] >= revenue_threshold).sum()
    total_count = len(director_movies)
    return round(success_count / total_count * 100, 1) if total_count > 0 else 0

# 기준 3: 수익 2억 달러 이상 (메가 히트 기준)
def get_mega_hit_rate(director_name, revenue_threshold=200_000_000):
    """메가 히트(2억불 이상) 성공률"""
    director_movies = df_full[df_full['director'] == director_name]
    success_count = (director_movies['revenue'] >= revenue_threshold).sum()
    total_count = len(director_movies)
    return round(success_count / total_count * 100, 1) if total_count > 0 else 0

# 기준 4: 평점 7.0 이상 (작품성 기준)
def get_quality_rate(director_name, rating_threshold=7.0):
    """작품성(평점 7.0 이상) 성공률"""
    director_movies = df_full[df_full['director'] == director_name]
    success_count = (director_movies['vote_average'] >= rating_threshold).sum()
    total_count = len(director_movies)
    return round(success_count / total_count * 100, 1) if total_count > 0 else 0

# 모든 기준 적용
director_stats['ROI3.0이상(%)'] = director_stats.index.map(lambda x: get_roi_success_rate(x, 3.0))
director_stats['블록버스터(%)'] = director_stats.index.map(lambda x: get_blockbuster_rate(x, 100_000_000))
director_stats['메가히트(%)'] = director_stats.index.map(lambda x: get_mega_hit_rate(x, 200_000_000))
director_stats['작품성7.0이상(%)'] = director_stats.index.map(lambda x: get_quality_rate(x, 7.0))

# 출력: 다양한 기준별 Top 20

print("\n\n" + "=" * 120)
print("=== ROI 3.0 이상 성공률 Top 20 (최소 5작품) ===")
print("=" * 120)
roi_analysis = director_stats[director_stats['작품수'] >= 5].sort_values('ROI3.0이상(%)', ascending=False)
print(roi_analysis[['작품수', 'ROI3.0이상(%)', 'ROI중앙값', '주요장르', '평균수익', '활동시기']].head(20).to_string())

print("\n\n" + "=" * 120)
print("=== 블록버스터(1억불) 성공률 Top 20 (최소 5작품) ===")
print("=" * 120)
blockbuster_analysis = director_stats[director_stats['작품수'] >= 5].sort_values('블록버스터(%)', ascending=False)
print(blockbuster_analysis[['작품수', '블록버스터(%)', '메가히트(%)', '주요장르', '총수익', '활동시기']].head(20).to_string())

print("\n\n" + "=" * 120)
print("=== 메가 히트(2억불) 성공률 Top 20 (최소 5작품) ===")
print("=" * 120)
mega_hit_analysis = director_stats[director_stats['작품수'] >= 5].sort_values('메가히트(%)', ascending=False)
print(mega_hit_analysis[['작품수', '메가히트(%)', '블록버스터(%)', '주요장르', '총수익', '활동시기']].head(20).to_string())

print("\n\n" + "=" * 120)
print("=== 작품성(평점 7.0) 성공률 Top 20 (최소 5작품) ===")
print("=" * 120)
quality_analysis = director_stats[director_stats['작품수'] >= 5].sort_values('작품성7.0이상(%)', ascending=False)
print(quality_analysis[['작품수', '작품성7.0이상(%)', '평균평점', '주요장르', '평균수익', '활동시기']].head(20).to_string())

# 종합 분석: 흥행 + 작품성 모두 우수한 감독

print("\n\n" + "=" * 120)
print("=== 흥행·작품성 균형 Top 15 (블록버스터 50%+ & 작품성 50%+) ===")
print("=" * 120)
balanced_directors = director_stats[
    (director_stats['작품수'] >= 5) & 
    (director_stats['블록버스터(%)'] >= 50) & 
    (director_stats['작품성7.0이상(%)'] >= 50)
].sort_values('총수익', ascending=False)

print(balanced_directors[['작품수', '블록버스터(%)', '작품성7.0이상(%)', '주요장르', '총수익', '평균평점']].head(15).to_string())